In [1]:
import os
import numpy as np
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

general code for webcam input from media pipe

In [ ]:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()

take photo with webcam and save the landmarks 

In [6]:
import cv2
import mediapipe as mp
import os
import pickle

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

def capture_hand_landmarks(num_photos):
    cap = cv2.VideoCapture(0)
    with mp_hands.Hands(
        model_complexity=0,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
        photo_count = 0
        while photo_count < num_photos:
            # Capture a frame from the camera
            ret, frame = cap.read()
            if not ret:
                continue

            # Display the frame
            cv2.imshow("Capture Frame", frame)

            # Check if the user wants to take a photo
            key = cv2.waitKey(1)
            if key == ord(" "):
                # Take a photo
                image_path = f"C:\\Users\\benja\Documents\\CV\\CV\\hand_commands\\images\\images_{photo_count}.jpg"
                cv2.imwrite(image_path, frame)

                # Get the landmarks of the hand
                image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = hands.process(image)
                if results.multi_hand_landmarks:
                    landmarks = results.multi_hand_landmarks[0]
                    landmarks_dict = {}
                    for index, landmark in enumerate(landmarks.landmark):
                        landmarks_dict[index] = (landmark.x, landmark.y, landmark.z)

                    # Save the landmarks to a file
                    landmarks_path = f"C:\\Users\\benja\\Documents\\CV\\CV\\hand_commands\\landmarks\\landmarks_{photo_count}.pickle"
                    os.makedirs(os.path.dirname(landmarks_path), exist_ok=True)
                    with open(landmarks_path, "wb") as f:
                        pickle.dump(landmarks_dict, f)

                print(f"Photo {photo_count + 1} and hand landmarks saved to disk.")
                photo_count += 1

        # Release the camera and destroy the windows
        cap.release()
        cv2.destroyAllWindows()


In [7]:
capture_hand_landmarks(2)

Photo 1 and hand landmarks saved to disk.
Photo 2 and hand landmarks saved to disk.


method to get the most similar landmark in a given directory   

In [1]:
def get_most_similar_landmark(directory, target_landmark):
    # Get a list of all the landmarks files in the directory
    landmarks_files = sorted([
        os.path.join(directory, filename) for filename in os.listdir(directory)
        if filename.startswith("landmarks_") and filename.endswith(".pickle")
    ])

    # Initialize variables to keep track of the most similar landmark
    max_similarity = -1
    most_similar_number = None

    # Iterate over each landmarks file
    for landmarks_file in landmarks_files:
        # Extract the number from the filename
        number = int(os.path.basename(landmarks_file).split("_")[1].split(".")[0])

        # Load the landmarks from the file
        with open(landmarks_file, "rb") as f:
            landmarks_dict = pickle.load(f)

        # Calculate the similarity between the target landmark and each landmark in the file
        similarities = []
        for landmark in landmarks_dict.values():
            similarity = np.dot(landmark, target_landmark) / (np.linalg.norm(landmark) * np.linalg.norm(target_landmark))
            similarities.append(similarity)

        # Get the maximum similarity and update the most similar landmark if necessary
        max_similarity_in_file = max(similarities)
        if max_similarity_in_file > max_similarity:
            max_similarity = max_similarity_in_file
            most_similar_number = number

    return most_similar_number


open webcam and pass frame to most similar method